## CS345 Fall 2022 Assignment 3


In [1]:
import math
import numpy as np
import pandas as pd
from numpy import genfromtxt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm

### Preliminaries

Datasets:

* The [QSAR](http://archive.ics.uci.edu/ml/datasets/QSAR+biodegradation) data for predicting the biochemical activity of a molecule.
* The [Wisconsin breast cancer wisconsin dataset](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer).
  

## Part 1:  choosing optimal hyperparameters

Just about any machine learning algorithm has some **hyperparameters**.  These are parameters that are set by the user and are not determined as part of the training process.
The perceptron for example, has two of those - the number of epochs and the learning rate.  For the k-nearest neighbor classifier (kNN) it's the number of neighbors, $k$, and for the linear SVM it's the soft margin constant, $C$.  Our objective in machine learning is to obtain classifiers with high accuracy, and have good estimates of how well they are performing.  In other words, we need to know how accurate a classifier would be on unseen data.  This is why we use separate test sets that the classifier has not seen for evaluating accuracy.

When working with classifiers with hyperparameters you may be tempted to apply the following procedure:

* Randomly split the data into separate train and test sets.
* Loop over a list of candidate values for the hyperparameter.
* For each value, train the classifier over the training set and evaluate its performance on the test set.
* Choose the parameter value that maximizes the accuracy over the test set, and report the accuracy that you obtained.

However, it turns out that this procedure is flawed, and the resulting accuracy estimate can be overly optimistic.  This is because the choice of the best performing parameter value used information about the test set: by selecting the best value we used information about the labels of the test set.  Therefore, the resulting accuracy estimate uses the labels of the test set, invalidating this accuracy estimate as being totally without any knowledge of the test set.

Here is a better approach.  Rather than splitting the data into train and test sets, we will now split the data into three sets:  **train, validation, and test**.  The validation set will be used for evaluation of different values of the hyperparameter, leading to the following approach:

* Randomly split the data into separate train, validation, and test sets (say with ratios of 0.5, 0.2, 0.3).
* Loop over a list of candidate values for the hyperparameter.
* For each value, train the classifier over the **training set** and evaluate its performance on the **validation set**. 
* Choose the best classifier, and report its accuracy over the **test set**.

Your task is as follows:

* Use the method described above to evaluate the performance of the kNN classifier over the QSAR and Wisconsin breast cancer dataset.  Use a wide range of $k$ values.  Repeat the process ten times over different train/test splits and report the average accuracy over the test set.  What value of $k$ was chosen?  Note that the optimal value of $k$ may vary for different splits.  Comment on your results.

* Perform the same experiment for the linear SVM. In this case the soft-margin constant $C$ is the hyperparameter that requires an informed choice.  Use a wide range of values for $C$, as we have done in class.  Comment on your results.

You may use the scikit-learn kNN and SVM implementations and `train_test_split`.

In [2]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.375, random_state=1)
T = np.linspace(0, 5, 500)[:, np.newaxis]

ks = [1,5,10,100,250]
for k in ks :
    knn = KNeighborsClassifier(k)
    knn.fit(X_train, y_train)    
    predicted = knn.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    print("kNN accuracy (k = %i) = %f" % (k, acc))



kNN accuracy (k = 1) = 0.903509
kNN accuracy (k = 5) = 0.929825
kNN accuracy (k = 10) = 0.921053
kNN accuracy (k = 100) = 0.859649
kNN accuracy (k = 250) = 0.631579


In general, the accuracy goes down as you increase k. However, when k = 5 the accuracy is the highest.

In [4]:
Cs = [1,5,10,100,250]
for C in Cs :
    classifier = svm.SVC(kernel="linear", C=C)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print('accuracy: ', np.mean(y_test == y_pred))

accuracy:  0.9473684210526315
accuracy:  0.9649122807017544
accuracy:  0.956140350877193
accuracy:  0.9473684210526315
accuracy:  0.956140350877193


The accuracy is higher than the kNN, it also follows a up and down pattern with the accuracy.

In [5]:
QSAT_URL = "http://archive.ics.uci.edu/ml/machine-learning-databases/00254/biodeg.csv"
QSATdata = pd.read_csv(QSAT_URL, sep=';', header=None)

def replace(x) :
    if (x=="RB") :
        return 1
    if (x=="NRB") :
        return 0
    else :
        return 0

y1 = QSATdata.iloc[:,-1]
y = np.zeros(y1.shape[0])

for i,txt in enumerate(y1) :
    if (txt=="RB") :
        y[i] = 1

QSATdata = QSATdata.iloc[:,:-1]

X = QSATdata.to_numpy()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.375, random_state=1)
T = np.linspace(0, 5, 500)[:, np.newaxis]

ks = [1,5,10,100,500]
for k in ks :
    knn = KNeighborsClassifier(k)
    knn.fit(X_train, y_train)    
    predicted = knn.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    print("kNN accuracy (k = %i) = %f" % (k, acc))

kNN accuracy (k = 1) = 0.791469
kNN accuracy (k = 5) = 0.800948
kNN accuracy (k = 10) = 0.758294
kNN accuracy (k = 100) = 0.710900
kNN accuracy (k = 500) = 0.625592


Accuracy also goes down as k increases

In [7]:
Cs = [1,5,10,100,250]
for C in Cs :
    classifier = svm.SVC(kernel="linear", C=C)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print('accuracy: ', np.mean(y_test == y_pred))

accuracy:  0.8483412322274881
accuracy:  0.8388625592417062
accuracy:  0.8388625592417062
accuracy:  0.8436018957345972
accuracy:  0.8246445497630331


The accuracy for C = 5 and C = 10 is the same. Also it follows the same pattern of going up and down as you change C. Acc

## Part 2:  PCA for removing noise from data

As we have seen in class, performance of the nearest neighbor classifer degrades when the data has noisy features that are not relevant to the classification problem.  To remedy this problem, we will use PCA to reduce the dimensionality of the data.

Here is your task:

* Use the QSAR dataset and evaluate performance of K nearest neighbors and SVM.  For simplicity, choose the values of K and $C$ that you selected in part 1.  Add 2000 noise features and evaluate model performance after doing so.
* Next, use PCA to represent the data in the space of a given number of principal components.  Evaluate the performance of the KNN and SVM classifiers as you vary the number of principal components (no need to go above the original dimensionality of the dataset when doing so).  Plot the accuracy of each classifier as you vary the number of components.
* Discuss your results:  was PCA useful for improving classifier performance?  Which of the two classifiers appears to be more robust to noise?
* In class your instructer noted that the data needs to be centered or standardized before applying PCA.  Do you observe a difference in classifier performance when you center or standardize the data before applying PCA?  (Recall that centering refers to subtracting the mean from each feature, making it so that each feature has a mean of 0).


### Your Report

Answer the questions in the cells reserved for that purpose.


### Submission

Submit your report as a Jupyter notebook via Canvas.  Running the notebook should generate all the plots in your notebook.

### Grading 

Although we will not grade on a 100 pt scale, the following is a grading sheet that will help you:


```
Grading sheet for assignment 3

Part 1:  45 points (5 pts for discussion).
Part 2:  55 points (10 pts for discussion).
```

Grading should be based on the following criteria:

  * Code correctness.
  * Plots and other results are well formatted and easy to understand.
  * Interesting and meaningful observations made where requested.
  * Notebook is readable, well-organized, and concise.
  
